# Best optimizer parameters:

### BUY ~ 74%
- PriceChange: low_price_quantile **10**
- LinearReg: timeperiod	**24**

# Parameter optimization

In [1]:
import sys
import pandas as pd
import numpy as np
from optimizer import Optimizer
from os import environ


# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

# Get configs
configs = ConfigFactory.factory(environ).configs


pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange', 'LinearReg']
opt_limit = 100
load = False

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i +  1) for i in range(10)]}, 
              'LinearReg': {'timeperiod': [16, 20, 24, 28, 32], 'low_bound': [0]}}
opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

try:
    open(f'opt_{"_".join(pattern)}_{ttype}.pkl', 'w').close()
except FileNotFoundError:
    pass
stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat = stat.sort_values(['pct_right_forecast_avg', 'forecasts_num'], ascending=False)
stat[stat['forecasts_num'] > 10].head(20)

Number of combinations is 50


  0%|          | 0/50 [00:00<?, ?it/s]

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg
17,PriceChange_LinearReg,4,24,0,100.0,66.67,66.67,75.00,66.67,66.67,66.67,66.67,66.67,66.67,66.67,58.33,75.00,83.33,83.33,83.33,75.00,75.00,75.00,75.00,83.33,83.33,75.00,75.00,1.79,2.35,2.22,0.84,2.69,2.31,3.21,4.53,3.59,4.07,4.24,4.68,4.49,4.58,5.39,4.99,6.21,7.04,6.84,6.80,6.69,6.51,6.07,6.90,12,73.958750
18,PriceChange_LinearReg,4,28,0,100.0,66.67,66.67,75.00,66.67,66.67,66.67,66.67,66.67,66.67,66.67,58.33,75.00,83.33,83.33,83.33,75.00,75.00,75.00,75.00,83.33,83.33,75.00,75.00,1.79,2.35,2.22,0.84,2.69,2.31,3.21,4.53,3.59,4.07,4.24,4.68,4.49,4.58,5.39,4.99,6.21,7.04,6.84,6.80,6.69,6.51,6.07,6.90,12,73.958750
48,PriceChange_LinearReg,10,28,0,100.0,73.68,73.68,78.95,68.42,68.42,68.42,73.68,68.42,68.42,73.68,63.16,73.68,78.95,78.95,78.95,73.68,68.42,68.42,68.42,73.68,73.68,68.42,63.16,1.76,2.11,1.14,1.37,2.35,2.60,3.30,2.55,2.54,2.83,2.64,3.68,5.47,5.47,5.37,4.30,5.84,6.28,6.61,6.05,5.83,5.38,4.84,6.49,19,72.805833
47,PriceChange_LinearReg,10,24,0,100.0,73.68,68.42,78.95,68.42,68.42,68.42,73.68,68.42,68.42,73.68,63.16,73.68,78.95,78.95,78.95,73.68,68.42,68.42,68.42,73.68,73.68,68.42,63.16,1.63,1.88,1.14,1.19,2.29,2.02,2.83,2.55,2.54,2.83,2.64,3.68,5.47,5.47,5.37,4.30,5.84,6.28,6.61,6.05,5.83,5.38,4.84,6.49,19,72.586667
43,PriceChange_LinearReg,9,28,0,100.0,70.59,70.59,76.47,64.71,64.71,64.71,70.59,64.71,64.71,70.59,58.82,70.59,76.47,76.47,76.47,70.59,70.59,70.59,70.59,76.47,76.47,70.59,70.59,1.63,2.11,1.14,1.10,2.35,2.02,4.87,6.03,5.71,5.83,5.77,5.77,5.68,6.17,6.01,5.68,6.57,7.79,7.08,7.01,6.96,7.40,7.31,7.31,17,71.570000
42,PriceChange_LinearReg,9,24,0,100.0,70.59,64.71,76.47,64.71,64.71,64.71,70.59,64.71,64.71,70.59,58.82,70.59,76.47,76.47,76.47,70.59,70.59,70.59,70.59,76.47,76.47,70.59,70.59,1.38,1.88,1.14,1.10,2.29,1.80,2.57,4.32,5.40,5.82,5.77,5.77,5.68,6.04,6.01,5.68,6.04,7.79,7.08,7.01,6.96,7.40,7.31,7.31,17,71.325000
28,PriceChange_LinearReg,6,28,0,100.0,66.67,66.67,73.33,60.00,60.00,60.00,66.67,60.00,60.00,66.67,53.33,66.67,73.33,73.33,73.33,66.67,66.67,66.67,66.67,73.33,73.33,66.67,66.67,1.63,2.15,1.14,0.79,2.29,2.02,1.55,1.75,0.35,1.98,2.46,3.34,3.30,2.99,4.77,4.30,5.84,6.28,6.61,6.05,5.83,5.38,4.84,6.49,15,67.778333
33,PriceChange_LinearReg,7,28,0,100.0,66.67,66.67,73.33,60.00,60.00,60.00,66.67,60.00,60.00,66.67,53.33,66.67,73.33,73.33,73.33,66.67,66.67,66.67,66.67,73.33,73.33,66.67,66.67,1.63,2.15,1.14,0.79,2.29,2.02,1.55,1.75,0.35,1.98,2.46,3.34,3.30,2.99,4.77,4.30,5.84,6.28,6.61,6.05,5.83,5.38,4.84,6.49,15,67.778333
38,PriceChange_LinearReg,8,28,0,100.0,66.67,66.67,73.33,60.00,60.00,60.00,66.67,60.00,60.00,66.67,53.33,66.67,73.33,73.33,73.33,66.67,66.67,66.67,66.67,73.33,73.33,66.67,66.67,1.63,2.15,1.14,0.79,2.29,2.02,1.55,1.75,0.35,1.98,2.46,3.34,3.30,2.99,4.77,4.30,5.84,6.28,6.61,6.05,5.83,5.38,4.84,6.49,15,67.778333
49,PriceChange_LinearReg,10,32,0,100.0,68.42,68.42,73.68,63.16,63.16,63.16,68.42,63.16,63.16,68.42,57.89,68.42,73.68,73.68,73.68,68.42,63.16,63.16,63.16,68.4